In [55]:
import pandas as pd
df = pd.read_csv("merged_forest_files_datas_2000.csv")
print(df.head())
df['high_risk'] = df['confidence'].apply(lambda x: 1 if x >= 80 else 0)
print(df['high_risk'].value_counts())


   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0   24.6078    88.1259       312.6   3.4    1.7  2000-11-02       544   
1   27.6587    90.3176       305.9   1.2    1.1  2000-11-03       448   
2   27.1644    91.2498       308.6   1.1    1.0  2000-11-03       448   
3   28.0703    90.1799       311.4   1.1    1.0  2000-11-08       506   
4   27.8122    90.6568       301.5   1.0    1.0  2000-11-10       454   

  satellite instrument  confidence  version  bright_t31   frp daynight  type  
0     Terra      MODIS           0      6.2       298.5  34.0        D     0  
1     Terra      MODIS          38      6.2       288.8   8.3        D     0  
2     Terra      MODIS          45      6.2       296.0   7.0        D     0  
3     Terra      MODIS          44      6.2       298.3  10.8        D     0  
4     Terra      MODIS          21      6.2       284.4   5.1        D     0  
high_risk
0    1922
1     142
Name: count, dtype: int64


In [63]:
import pandas as pd
df = pd.read_csv("merged_forest_files_datas_2000.csv")
print(df.head())
df['high_risk'] = df['confidence'].apply(lambda x: 1 if x >= 80 else 0)
print(df['high_risk'].value_counts())


   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0   24.6078    88.1259       312.6   3.4    1.7  2000-11-02       544   
1   27.6587    90.3176       305.9   1.2    1.1  2000-11-03       448   
2   27.1644    91.2498       308.6   1.1    1.0  2000-11-03       448   
3   28.0703    90.1799       311.4   1.1    1.0  2000-11-08       506   
4   27.8122    90.6568       301.5   1.0    1.0  2000-11-10       454   

  satellite instrument  confidence  version  bright_t31   frp daynight  type  
0     Terra      MODIS           0      6.2       298.5  34.0        D     0  
1     Terra      MODIS          38      6.2       288.8   8.3        D     0  
2     Terra      MODIS          45      6.2       296.0   7.0        D     0  
3     Terra      MODIS          44      6.2       298.3  10.8        D     0  
4     Terra      MODIS          21      6.2       284.4   5.1        D     0  
high_risk
0    1922
1     142
Name: count, dtype: int64


In [64]:
from sklearn.model_selection import train_test_split
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
target = 'high_risk'
df = df.dropna(subset=features + [target])
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 1651
Testing samples: 413


In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy Score: 0.9636803874092009

Confusion Matrix:
 [[371   1]
 [ 14  27]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       372
           1       0.96      0.66      0.78        41

    accuracy                           0.96       413
   macro avg       0.96      0.83      0.88       413
weighted avg       0.96      0.96      0.96       413



In [ ]:
from sklearn.preprocessing import StandardScaler
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
scaler = StandardScaler()
X = df[features]
X_scaled = scaler.fit_transform(X)
df['predicted_risk'] = model.predict(X_scaled)
df['risk_level'] = df['predicted_risk'].map({1: 'High', 0: 'Low'})
df.to_csv("fire_risk_predictions.csv", index=False)

print("fire risk prediction saved in the file")


Fire risk predictions saved to fire_risk_predictions.csv


C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [67]:
df['predicted_risk'] = model.predict(pd.DataFrame(X_scaled, columns=features))


In [68]:
import folium
from folium.plugins import HeatMap
high_risk_df = df[df['risk_level'] == 'High']
m = folium.Map(location=[22.9734, 78.6569], zoom_start=5)
heat_data = [[row['latitude'], row['longitude']] for _, row in high_risk_df.iterrows()]
HeatMap(heat_data, radius=7).add_to(m)
m.save("high_risk_fire_heatmap.html")
m


In [69]:
high_risk_df = df[df['risk_level'] == 'High']
top_risky_areas = high_risk_df.sort_values(by=['brightness', 'frp'], ascending=False).head(10)
top_risky_areas.to_csv('top_10_high_risk_fire_locations.csv', index=False)
top_risky_areas[['latitude', 'longitude', 'brightness', 'frp', 'confidence', 'acq_time']]


,latitude,longitude,brightness,frp,confidence,acq_time
1319,26.3998,95.0148,467.6,2975.6,100,410
1314,26.4137,95.0063,431.2,1531.9,94,410
1316,26.3992,95.0232,428.9,1448.4,100,410
1317,26.4143,94.9992,421.7,1258.4,94,410
1521,26.4018,95.0714,409.4,970.6,100,506
1532,26.5083,95.1787,406.2,459.6,100,453
1113,26.5411,95.0771,402.9,320.4,100,447
1534,26.5123,95.1722,399.3,390.9,100,453
1519,26.4018,95.0759,395.7,708.2,100,506
1284,26.4058,95.0556,394.1,674.6,100,506


In [70]:
high_risk_df = df[df['risk_level'] == 'High']
top_risky_areas = high_risk_df.sort_values(by=['brightness', 'frp'], ascending=False).head(10)
top_risky_areas.to_csv('top_10_high_risk_fire_locations.csv', index=False
top_risky_areas[['latitude', 'longitude', 'brightness', 'frp', 'confidence', 'acq_time']]


,latitude,longitude,brightness,frp,confidence,acq_time
1319,26.3998,95.0148,467.6,2975.6,100,410
1314,26.4137,95.0063,431.2,1531.9,94,410
1316,26.3992,95.0232,428.9,1448.4,100,410
1317,26.4143,94.9992,421.7,1258.4,94,410
1521,26.4018,95.0714,409.4,970.6,100,506
1532,26.5083,95.1787,406.2,459.6,100,453
1113,26.5411,95.0771,402.9,320.4,100,447
1534,26.5123,95.1722,399.3,390.9,100,453
1519,26.4018,95.0759,395.7,708.2,100,506
1284,26.4058,95.0556,394.1,674.6,100,506


In [72]:
import joblib
joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [75]:
import pandas as pd
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')
fire_df = pd.read_csv("merged_forest_files_datas_2000.csv")


In [76]:
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
X = fire_df[features]


In [77]:
X_scaled = scaler.transform(X) 
predictions = model.predict(X_scaled)


C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [78]:
fire_df['risk_level'] = predictions
fire_df['risk_level'] = fire_df['risk_level'].map({0: 'Low', 1: 'High'})


In [81]:
import pandas as pd
import joblib
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')
fire_df = pd.read_csv("merged_forest_files_datas_2000.csv")
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
X = fire_df[features]
X_scaled = scaler.transform(X)
predictions = model.predict(X_scaled)
fire_df['risk_level'] = predictions
fire_df['risk_level'] = fire_df['risk_level'].map({0: 'Low', 1: 'High'})

fire_df.to_csv("fire_with_risk_level.csv", index=False)


C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [82]:
import pandas as pd
import joblib
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')
df = pd.read_csv("merged_forest_files_datas_2000.csv")
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
X = df[features]
X_scaled = scaler.transform(X)
predictions = model.predict(X_scaled)
df['risk_level'] = predictions  # 0 or 1
df['risk_level'] = df['risk_level'].map({0: 'Low', 1: 'High'}) 
risk_scores = model.predict_proba(X_scaled)[:, 1] 
df['risk_score'] = risk_scores
df.to_csv("fire_data_with_risk.csv", index=False)


C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [83]:
import pandas as pd
import joblib
model = joblib.load("model.pkl")
scaler = joblib.load("scaler.pkl")
df = pd.read_csv("merged_forest_files_datas_2000.csv") 
features = ['latitude', 'longitude', 'brightness', 'bright_t31', 'frp']
X = df[features]
X_scaled = scaler.transform(X)
df['risk_level'] = model.predict(X_scaled)
df['risk_level'] = df['risk_level'].map({0: 'Low', 1: 'High'})
df['risk_score'] = model.predict_proba(X_scaled)[:, 1]
df_sorted = df.sort_values(by='risk_score', ascending=False)
df_sorted.to_csv("fire_data_with_risk.csv", index=False)
print("prediction complete .saved fire_data_eith_risk")


Prediction complete. Results saved to 'fire_data_with_risk.csv'.


C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\tapas\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [84]:
pd.read_csv("fire_data_with_risk.csv")


,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,risk_level,risk_score
0,26.3998,95.0148,467.6,2.2,1.4,2000-12-27,410,Terra,MODIS,100,6.2,319.5,2975.6,D,0,High,0.981416
1,26.4137,95.0063,431.2,2.2,1.4,2000-12-27,410,Terra,MODIS,94,6.2,306.4,1531.9,D,0,High,0.976225
2,26.4143,94.9992,421.7,2.2,1.4,2000-12-27,410,Terra,MODIS,94,6.2,302.3,1258.4,D,0,High,0.975506
3,26.3992,95.0232,428.9,2.2,1.4,2000-12-27,410,Terra,MODIS,100,6.2,307.5,1448.4,D,0,High,0.971931
4,26.4018,95.0714,409.4,2.2,1.4,2000-12-26,506,Terra,MODIS,100,6.2,299.1,970.6,D,0,High,0.969654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,27.1765,94.2456,309.8,1.1,1.0,2000-12-23,435,Terra,MODIS,65,6.2,299.0,6.0,D,0,Low,0.346881
2060,26.7185,93.3716,310.1,1.3,1.1,2000-12-19,459,Terra,MODIS,0,6.2,299.5,11.5,D,0,Low,0.346500
2061,22.8368,94.7373,316.5,1.0,1.0,2000-11-12,443,Terra,MODIS,0,6.2,304.3,10.7,D,0,Low,0.344470
2062,23.7323,86.4396,317.9,1.0,1.0,2000-11-08,507,Terra,MODIS,57,6.2,306.3,7.4,D,0,Low,0.342665
